# 1. Data Preprocessing

## 1.1 Import Libraries

In [1]:
import time
import random
import numpy as np 
import pandas as pd 
from surprise import accuracy, Dataset, Reader, KNNWithMeans, SVD, BaselineOnly, Trainset
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from collections import defaultdict

In [2]:
#pip install surprise

## 1.2 Load Data

In [5]:
fname = r'Software'

In [6]:
random.seed(123)
mydata=pd.read_csv("/home/ubuntu/Notebooks/ratings_data/{}.csv".format(fname),names = ['item','user','rating','timestamp'])
del mydata['timestamp']
#reindex the column positions to be suitable for surprise
columns_titles = ['user','item','rating']
mydata=mydata.reindex(columns=columns_titles)
mydata.head()

,user,item,rating
0,A240ORQ2LF9LUI,0077613252,4.0
1,A1YCCU0YRLS0FE,0077613252,4.0
2,A1BJHRQDYVAY2J,0077613252,1.0
3,APRDVZ6QBIQXT,0077613252,3.0
4,A2JZTTBSLS1QXV,0077775473,5.0


## 1.3 Prepare data and check the number of users and items

In [7]:
# create a reader object to parse the dataset in the format appropriate for Surprise
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5))
# load the dataframe with reader as Surprise dataset object
data = Dataset.load_from_df(mydata, reader)
#with full dataset as training set
trainsetfull = data.build_full_trainset()
#check the unique items, users and ratings
print('Total number of users: ', trainsetfull.n_users,)
print('Total number of items: ', trainsetfull.n_items,)
print('Total number of ratings: ', trainsetfull.n_ratings,)
#print(mydata.user.nunique())
#print(mydata.item.nunique())
#print(mydata.rating.nunique())

Total number of users:  375147
Total number of items:  21663
Total number of ratings:  459436


# 2. Apply Collaborative Filtering using Sequential Algorithm

# 2.1 Memory based approach (item-based)

### Use Centered KNN to calculate similarity and check RMSE, MSE, MAE scores for test set

In [8]:
##############################################################
#memory based approach
##############################################################

#################
#item based
#################

#to use item-based cosine similarity
sim_options_item = {
    "name": "cosine",
    "user_based": False,  #compute similarities between items
}
#choose centered KNN as the algorithm for finding similar items
algo_item = KNNWithMeans(sim_options=sim_options_item)
#build the training set
trainset, testset = train_test_split(data, test_size=.20)
print('Number of users in trainset: ', trainset.n_users,)
print('Number of items in trainset: ', trainset.n_items,)
print('Number of ratings in trainset: ', trainset.n_ratings,)
#train the algorithm on the trainset, and predict ratings for the testset
algo_item.fit(trainset)
predictions_item = algo_item.test(testset)
#then compute RMSE, MSE and MAE
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_item)
accuracy.mse(predictions_item)
accuracy.mae(predictions_item)

Number of users in trainset:  308733
Number of items in trainset:  20101
Number of ratings in trainset:  367548
Computing the cosine similarity matrix...
Done computing similarity matrix.
The testing accuracy in RMSE is: 
RMSE: 1.5825
MSE: 2.5042
MAE:  1.3512


1.351183055488626

In [9]:
#for converting between raw (as strings, in the original data) 
#and inner IDs (integers, as converted by the surprise package itself)
#trainset_iids = list(trainset.all_items())
#iid_converter = lambda x: trainset.to_raw_iid(x)
#trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [10]:
#for cross validation, not necessary for our project
#results = cross_validate(
#    algo = algo_item, data = data, measures=['RMSE','MSE','MAE'], 
#    cv=5, return_train_measures=True
#    )
#print(results['test_rmse'].mean())
#print(results['test_mse'].mean())
#print(results['test_mae'].mean())

In [11]:
#takes long
#anti_testset = trainsetfull.build_anti_testset()

In [8]:
#takes way too long!
#predictions_antitest = algo_item.test(anti_testset)

### Check the estimated ratings on the testset

In [5]:
#estimated ratings on the testset
pred_df = pd.DataFrame(predictions_item)
pred_df[['uid', 'iid', 'est']]

,uid,iid,est
0,A2YV8GR7IWAQ41,B00GOLGWVK,4.939951
1,AWFVKSW9BSTSX,B00BXLVAD6,5.000000
2,A2N3BP1J6X8WW3,B01E4QQ6QM,4.746171
3,AABEGIPYNQAEA,B009110O8C,4.937968
4,APY7B69YN6BUC,B0091JKJ0M,4.674095
...,...,...,...
29434,A1J4O0VR3G4D3I,B00A4EK106,4.674095
29435,A1O6VJM0XQ4K8C,B004LLILM8,4.674095
29436,A1GSJ508Q1SKZC,B00BXLTR6S,4.674095
29437,A1RMZXDJ0PC2B0,B00CHSWJPC,4.674095


### Check the top recommendations for each user

In [6]:
#see the predictions on the test dataset
pred_df = pd.DataFrame(predictions_item)
del pred_df['details']
#finding recommendation for a user
pred_df[(pred_df['est']>0.5)&(pred_df['uid']=='APV13CM0919JD')]

#getting top 10 recommendations for each user
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions_item, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A2YV8GR7IWAQ41 ['B00BXLVHZM', 'B0091JKY0M', 'B00GOLGWVK']
AWFVKSW9BSTSX ['B00BXLVAD6']
A2N3BP1J6X8WW3 ['B01E4QQ6QM']
AABEGIPYNQAEA ['B009110O8C']
APY7B69YN6BUC ['B0091JKJ0M']
AHWF5KD4X61DB ['B00UXLIUM6']
A3KNX9T8VEWX1D ['B004Q7CK9M']
A2HNR418LK01KH ['B007V6ETDK']
A2YLM5Z2EX9COB ['B0091JKY0M']
A3AEPGBNN78HSK ['B0091JKVU0']
A1DYU9OGB3XEXN ['B00F56PFYK']
A1YQTS9D9YJ8PX ['B004LLIL4G']
A63HW9AIN5MI3 ['B01B25L7US', 'B01FERQPAM', 'B00ADR2XIC']
A2QEDZ3PDSNQ0 ['B007HR1E2W']
A115L2GCK88XDV ['B006PJHP62']
A12F6RRZDS38JC ['B00PG4FAH8']
ASRIW7IX6L4XW ['B00CHSWKEM']
AWKLZSYFU305D ['B00YD74M4Q']
A19DHJWP7C7B9O ['B004Q7CK9M']
A2SJQOF0MO7MNP ['B004Q7CK9M', 'B00BXLV32O']
A9STZHEZ84W4W ['B00H5BNENW']
A3FE43YP9EH1KA ['B005ESMJ02']
A1Y3DIH2HL4OTW ['B00ET5WBU4']
A2V33MPVKCAB1Q ['B0145WHXGM']
A2AX1046M7Q9UV ['B00FTGTIOE']
A2B4DHYEYT6RO5 ['B00XUUAB92']
A1DQUE5ESEMMSQ ['B00MV9FQXO']
A1S37KD9WKAJLE ['B00PMLD91O']
A3RDW67C8KTFRR ['B00BXLVAD6']
A2YGZV9S9F31UO ['B00H5BMH44']
A2YJM56DIQC58C ['B015WY0DOQ']
A8OTPRZCI

AQ2XE6L5KHLV6 ['B004LLIL5A']
A2LZN943ROJL63 ['B0091JKY0M']
A3R1QOG5TUBSRR ['B0091JKY0M']
A2MJYRJSZ3CM0G ['B01FERQPAM']
A3PUXUT9GLH4C8 ['B00A4EK69C']
A1LL72IDQO2DHV ['B00C5UMCIG']
A1NADKSHMF3S6K ['B0091JKJ0M']
A3CBC49HZYDDB1 ['B004Q7CK9M']
A3H5CWJ3ADVXVS ['B014S24DAI']
ABDH36OJAH7O2 ['B00H5BN6V2']
A13PHRXGQIQZU5 ['B006PJHP62']
A11DDUTSLAM4HF ['B004KNWX6M']
A2VZCTX5RHHFM9 ['B01E4QUN0W']
AGXPY1GUS73VI ['B005ESMJ02']
A39KKN4JTMEHE3 ['B0091JKJ0M']
ARP9JWMBJLP4E ['B00JDQKFPC']
A1PAY3861H305X ['B01E4QS95I']
A3KSVN7U8GOBXZ ['B00BT1XFBU']
A1QE4KAEGRKDZC ['B004Q7CK9M']
ATHKA4IE3COCV ['B004LLILGO']
A3L51DY6HO8J39 ['B00B2TFSO6']
A3PL5873CW6JZ0 ['B00MV9P8MS', 'B00911NGTG']
A0807760H1KAYIBFPIS ['B004LLIKY2']
A3DERU757241SI ['B007V6ETDK']
A3BHSHYMD1B73P ['B004KNWWOK']
A10A60MOP7949C ['B004Q7CK9M']
A1O6XKT8YHRDYY ['B0091JKVU0']
A2ABVOB6RVKNUU ['B006PJHP62']
AL2AIAXA090ZI ['B01E4QS95I', 'B00BXLVFAO', 'B00BXLT3VC']
A1Y3WJQ1B35P4E ['B004Q7CK9M']
A3Q6SV7HOQRXMI ['B004LLIL5A']
A36YC5QFXBQ1LI ['B004LLILPU']

A2GIIGRTTGX5ED ['B0189TCA7E', 'B00MV9LSHC']
A1VUNFXQ89QW3C ['B00I5428EE']
AW8AUUTP0JP52 ['B00G4IV2VI']
A1MZ8L7FT8P2BW ['B00H5BMH44']
A1ZYP1Y0R1EMEZ ['B004W8D102']
AO2A6QO4WP922 ['B004LLIKVU']
A14Q5RGZ5OEGN2 ['B00BWDH4DK']
A2V8V78VJ8Y6ZN ['B00VF0MCCS']
A1OXSMCPZBDE7O ['B005EISPLE']
A1TPO5PADDXE3J ['B00G4IWEZG']
A2ZUPTBJ9SQCX ['B004LLJ6LI']
A2YXWIB141OYIG ['B00ADR2XIC']
A1Y6VQYNCT53A2 ['B005ESMFP6']
A3UC3KA1VE7KXV ['B014S24DAI', 'B004Q7CK9M']
A1MLIB97IRIRZM ['B0091JKVU0', 'B00CHQ7I2S']
AOHZMPEGSQXWA ['B004LLILL4']
A2WT72O2V4FYG0 ['B00G4IWDXE']
A1EW6Q74DAR0M ['B004KNWWU4']
A304AY9M0FAI3Y ['B0062ONJ6U']
A24ISEVWULRQXW ['B00FTJI60I', 'B005DHN6E2']
A3H2L2WLJH77H5 ['B004KNWWR2']
A1GOJY3Y7LL2Y6 ['B00BXLSZPM']
A75Y65SJDSODV ['B0091JL3OI']
A3DQQFJDRH5EST ['B00BXLUQ6S']
A2MLBN3XZ7QKLS ['B01GKWEPBG']
A1LWWQQI1VLMKB ['B004Q7CK9M']
A3PK9ZKQU7T38B ['B00H5BN6V2']
ARIGF8JKKYAW1 ['B0080IR6BA']
A2N5O6LJSJ89DZ ['B00BXLUO1U']
A3INE7M0AEDTPL ['B00H5BMH44']
AZVZS872UCDAJ ['B018UPUQ5A']
A3J1C3TMUMN00P ['B00EL

A1YWLO2PB24Q1T ['B0091JKJ0M']
AJR5YI0AJK93J ['B00K4PIXXW']
A1MYM0L4TRK6PZ ['B005ESMMWW']
A1W7HICRRY6YUO ['B004KNWWTA']
A29NFL3SK1XO1N ['B0091JKVU0']
A2D29CI8CY9SR9 ['B007EE5I6G']
A1OCPFK1WY1EDY ['B00H5BM0K0']
A325ATR42MDX95 ['B00MV9FQXO']
A1E1ALE46KSY96 ['B0091JKVU0']
AGL1ZGOFALNRZ ['B00PG8502O', 'B005ESMFP6']
A2SXY8VY3F23OW ['B0091JKJ0M']
A1OL2KSIQUK3YD ['B01G44QN9W']
A1QR9SNETH3MDH ['B01DCN57F4']
A2YZGVPUIZXTIJ ['B004KNWWPE']
A2BPI1PX1UTYMK ['B002XNLC04']
A3J2YLYS3JOGRZ ['B00JDQJZWG']
A126OO6QB06IPH ['B00MV9GCYQ']
A13KC62PX2TV3P ['B004LLIKVU']
A1GC611X9O8T1A ['B01E4QUN0W']
A3FV6RIBR78N8U ['B00KV30IHG']
AV4COUDS1JD4H ['B0091JL3OI']
A2NT888PP7NAUC ['B00GOLGWVK']
A11XZFOBZ3DT2V ['B004LLIL32', 'B004KNWWU4']
A2VDMI1LWQ6R1M ['B00B2TFKHG']
A1A707ZSO58TEQ ['B01E4QUN0W']
A2Z18IVMU9EU8T ['B0091JL3OI', 'B004RD8XPK']
A3QG6OA3KJUASE ['B01E4QUN0W']
A1U9NOVNY6YK7Z ['B016BCKQXG']
A223DHIG1WXTU0 ['B01E4QUN0W']
AU8RGLT82799M ['B004KNWWU4']
A2F1INWG8T2S65 ['B004Q7CK9M']
AA810598ZVCXK ['B014S24DAI']
A4Y

A2XDAOYAS2R48Q ['B0091JKVU0']
A2904E2CUL54SN ['B00C5UMCIG']
A1Q8II8NWE74K4 ['B004LLIKVU']
A2VJWUXTPY2WI2 ['B00BXLURUI']
A149OMDE7UFZJV ['B00H5BMVG8']
A1UXJ2R6MJRJJR ['B004LLIL5A']
AKT3Z3O3T26PL ['B00BXLTZ6K', 'B00BXLUFQ4', 'B00BXLW5QC', 'B00BXLSZPM', 'B00YD743CW', 'B00F2RZMEA']
A2XXMPSDBQHJVM ['B00PG8502O']
A1HDSM89QGX1RD ['B0062ONF64']
ADCMPWUBRTNIS ['B004Q7CK9M']
AW3K7SA79ZVCR ['B00895H7FO']
AVGG2IM4NUS9F ['B0091JKVU0']
A1Y0A7W7O9LFH4 ['B004KNWX6C']
ALN4ZANYJF2EE ['B0145WHTES']
A3V7GEXJ47VY6O ['B00ISCEAMG']
A1J6862QDXS3HG ['B00B2TFSO6']
A2AXOCIAMG9AGE ['B004LLIKVU']
A2GF2GDMJKVCSQ ['B004Q7CK9M']
AEJXGSJXM3P5R ['B01H5PPJT4']
A3UEFXJPNO78T ['B005ESMMWW']
A2UH2OP15L3R0 ['B00CHSWGR8']
A26N2YLUBA9MLH ['B00ADR2XIC']
A23U0IPY57EJ2G ['B0066AZGJI']
A20CMSF8TQ8KU5 ['B018UPUQ5A']
A26YV2VVK89RMT ['B0145WHG5K']
A1DCXJ9TFQIYFD ['B0187SWDSI']
A1WOPJ6MJBL1BF ['B00MV9GM8W']
A32KU9BE3ULHCL ['B00CHQ7I2S']
A322T4KQ562L1C ['B00H5BMF00']
A26K65J40SVG62 ['B004LLIL5A']
A27HDNYTF15WQA ['B00CHSWG6O']
A3PUQBD0

...

A112M18TIOVRU5 ['B004Q7CK9M']
A8291WP1C6YX6 ['B01E4QPDV6']
A15ZHR29NLIHDF ['B00B2TFURQ']
A2OVFX8PJ2YVD ['B0078EPRPE']
ADLVAD4JI8EYQ ['B004LLIL4G']
A2SPQ8BZDT36C5 ['B007V6ETDK']
A3ER6ARJ23CIN8 ['B00H5BPOKI']
A1C9HZOWY3W338 ['B004Q7CK9M']
A2LT3KVAFW87MQ ['B0091JKJ0M']
A1WFAQFP7U9EFZ ['B00MV9GGEC']
A3FAO74L0Q61AX ['B0062ON91K']
A2731QECLSA90W ['B00FTGTIOE']
A5MRPP9KDW1B5 ['B004LLIL5A']
A1XGWODYL02K05 ['B00CHSWGFU']
AFNL11V3PS172 ['B005ESMMWW']
A3RAYTM4WY0SFX ['B0091JKY0M']
A1KMOEDF4ELQDQ ['B00G4IWEZG']
A1TZWECB1JZLKY ['B0145WHTES']
A2X3SVS3OJS79C ['B006PJHP62']
A307OQNLT7L7L6 ['B00JDQKFPC']
A231J94N4SV61G ['B004LLIKVU']
A3F8FRMQQ81PC3 ['B018UPUQ5A']
A1XLRY7D93FDC2 ['B00MV9GRNW']
AKY2NMZ4HPD4H ['B01CZ54L66']
A2O2I8GXSTAOY6 ['B00CHSWG6O']
A2XBWF43VO66SK ['B005EISPLE']
APHUOXCRIKS75 ['B00PMLD91O']
A36USMOAXH22VR ['B005EISPLE']
A29HMBT94LUX9Y ['B004LLIL4G']
A1CPAD29P3G2JT ['B0066AZGD4']
A3380D2X8W4YKB ['B00BSXWO52']
A13BR4NT2880O1 ['B00A4EK106']
A31PJ5Z9TCV8Z7 ['B00BSXWOAC']
A6C0PDWS0Y8S4 ['B

A3SH0B4OTT9LLA ['B004LLILO6']
A146SXVPGFT7TR ['B01E4QUN0W']
A2M3ZZDOPTPGDJ ['B00B2TF61G']
A3DQ6V6190ZSXM ['B004Q7CK9M']
A1FZ6OH0J1H89I ['B00G4IWEZG']
AH7SZZLIFGS9Y ['B004KNWWV8']
A2LHY6EV3N4JZL ['B00BT1XFBU']
A1ZT4KRFXTA4W9 ['B00AR51Y5I']
ADGL3OS9T0S8W ['B00CHSWG6O']
A1N6EN2T72QLEH ['B00CHSWG6O']
A3AJ6PRTJWYXY3 ['B0091JKJ0M']
A137WX873B3V14 ['B00CHSWEJI']
A2O30TWLOGKD2C ['B006PJHPV2']
A36I61R3SM13Z4 ['B00MV9M0UQ']
A3GKDDJ1RN4I8X ['B004LLIKVU']
A15ZV2Y738VB0G ['B004KNWWOK']
A33QILRICUEV2A ['B004RD8XPK']
A2B74VVUK6F3MS ['B00UXLIUM6']
A2T4NGK4KHMB0O ['B007EE5KKK']
A1619ZJBQUDSD2 ['B0066AZGD4']
A2IAHDWN8UVL7X ['B0091JKJ0M']
AJU6XXW8H6NRG ['B004LLIL4G']
A12SO3HDCN6J24 ['B0066AZGD4']
A2YP3D1QF5U5ZE ['B00BWDH2O6']
A7QN3YS1SUO7G ['B018T7WFNA']
A2BAQEIKCC9IQY ['B004Q7CK9M']
A1TYOFI77SDS9Y ['B00BXQIGA6']
A30HOMSSF1BF1V ['B004Q7CK9M']
A3SIFQ3NO5XZLG ['B0080IR6BA']
A2XU36NTBV8X4M ['B004LLIL2I']
A23KZRIQBO8XT3 ['B004KNWX12']
A39AFT0386GV8H ['B01FYDFH5U']
A1TO1ZWEQ5NZKX ['B00H5BMH44']
A3C5WM6RY1OL0V

A1D1NVK6S6S3T1 ['B0187SVQNG']
A3PZW21UNBNNN4 ['B01E4QUN0W']
A2HMRUGW4QSWNI ['B01E4QRAX0']
A3FNFC0116353L ['B0091JL3OI']
A2ZQT39HSSHYMH ['B00MV9M8PS']
A3JCQ5IKQ6PUZN ['B004KNWWU4']
A1ADXNUPA5SK0V ['B00H5BMH44']
AGHOP6YMGUQ82 ['B00BXLVD8I']
A2X616HR0FF4KK ['B004Q7CK9M']
A2EVHOO5DG2969 ['B00E1QB6ZC']
A2EO692W3TZIQI ['B00JDQKFPC']
A2ZXKF3UFNW0XU ['B00H5BMHS0']
A3QJZBMR1DPNDI ['B004RD9ACA']
A3CDKLPKOZB93C ['B0091JKY0M']
A2GRAFAP3XQIXP ['B00BXQIGA6']
A271CD6EEYZKF9 ['B00BT1XEZM']
A26FROON8GGXYL ['B004LLIKY2']
A3TRUWR1HWTCY2 ['B0145WHUP6']
AJWGNTOF7QX7P ['B004LLILKK']
A1P8HM1CGFDNF2 ['B01CT5PHJ2']
A2QAQTK1SDV95B ['B01FERR9FW']
A20NMV4XA06RQC ['B00CRQ4AYW']
A2Y971BPIPJQ0S ['B005ESMMWW']
AK4V0NRW78DYG ['B0091JKVU0']
A2YTLNRVBC6SY6 ['B004LLIKY2']
ALYDJ18H4FIWL ['B004W8D0Y4']
A70GB8RXGKOSP ['B0091JKPDI']
A4MWLXXAFTVKB ['B00GU2ZZ8Y']
A1KA5AL98NH9DF ['B00NM4II5A']
A29HNS0XAPA5D0 ['B00CXZPG0O']
A34AJ41JDJRGQL ['B00AF0K90Q']
A182NX50N1CWNU ['B00X4RERY6']
A3HB8T2BS0D44B ['B004LLILGO']
A1VEELTKS8NLZB [

A3NTZN4UZFBNVU ['B004Q7CK9M']
A3S1L7B092HGSW ['B004KNWWV8']
A3HOIXUHJUJ1EW ['B00H5BMH44']
A2H8HBRV8CKLH9 ['B004KNWX6C']
A1ZW7GAGSHK5RI ['B00B2TFSO6']
A2SKK3XRGCZHNA ['B00Q5BN3P2']
ACNQI6RTN3IAD ['B0091JKPDI']
A3JD3X5PT081PW ['B00CHSWG6O']
A37SNJ68GROE0C ['B004W8D102']
A1VE7KWZBF14SP ['B018T7G9XM']
A375ROZEUOJ67C ['B00MV9P8MS']
A139C0HGV4UYCR ['B004LLIKY2']
A3JNCRQOQ7U4ND ['B01FERQT4E']
A149JJYFFDTUS ['B00BXLW5QC']
A1NJ8ISHV1C9TJ ['B00M0AHQSC']
AITEIH9VANMIG ['B00JDQJZWG']
A44YMQFQDYZIV ['B00A4EK69C']
A32ZMB5OE6BTRX ['B004Q7CK9M']
A26YOGG9AF3JVX ['B0091JKVU0']
A38IGV3B7NXI6M ['B014S1KPKG']
A1EXDOIWO5RAP2 ['B00JDQJZWG']
ANX51171V6CF2 ['B00BXLTR6S']
A2UABHP8Y4OYM ['B00BXLTUBA']
A3PAPW1NKQXCR1 ['B004LLILGO']
A20JVQ09AQ0Y9E ['B01E4QPDV6']
AC1QZ7CTEUM79 ['B01E4QUN0W']
A3NJZ9WMDVDSA4 ['B005DHN6E2']
A3951FWYFPYVN8 ['B00H5BMYAG']
AQD3U5UEZNVOY ['B0078EPRPE']
A4YPH4M6FBEN1 ['B00MV9GGEC']
A7PQEDE0HQVPE ['B00G4IV2VI']
A3981ELWLHGALB ['B004W8D102']
A2E6PGQQZ1UW2E ['B004LLILG4']
A1PARDKXN3UICG ['B00

A9IVN4GVB29SF ['B004LLIKVU']
A1P8M79QKCTNHG ['B00H5BN162']
AM70JEQRBJBFW ['B004LLIKVU']
A1RXUW3KBNK105 ['B004W8D0Y4']
A2L6ND4CH3ES93 ['B004LLIKVU']
A34GI9ZA48ONAL ['B0091JKVU0']
A3FMUMU77YZ7WN ['B00BXLTZ6K']
AJZ63WC8UC1QM ['B00GOLH84U']
A3LQ8K1IGLQ2WZ ['B00CRQ4CF4']
A2126MEAOW42QY ['B0145WHSDA']
A3T277HYNK80PM ['B00JQKK3JS']
ALJQR5CHQ2PJP ['B014S24DAI']
ATT941IUP9ZQ4 ['B00H5BMH44']
AG211D0FOEPUC ['B001GXRQW0']
A8170ZXBEP21G ['B00HI3QRZE']
A1Z9XLTNI48Q1E ['B00BXLW5QC']
A1CO22VQTT8QNJ ['B00PMLD91O']
APK9X8YLQWAV3 ['B00NU7UVUO']
A1CAVAQZPDMPEW ['B00BXLUI5M']
A2MDJC72AFJZ3B ['B00CT7B2KO']
A12UMEKY9SLOQ0 ['B004W8D0Y4']
AW7KVUU60H1XK ['B0066AZGD4']
A16UFYY7TVV9HI ['B004KNWX3U']
AY4IZ1BBJ05X9 ['B00B2TFSO6']
A3GQS4DHS1PY86 ['B004Q7CK9M']
A15M0YCRGD79EH ['B004LLIKVU']
A26ZWI6N523E67 ['B00HI3QVPA']
A1N9C63KT0YSSY ['B00MV9GCYQ']
A29I8B13K8A22 ['B005EISPOG']
AZY5WTW6SEMGY ['B0145WHUP6']
A1GM121YLFEV61 ['B00X6G8J3A']
A2BJF4X1EKY40M ['B004LLIL4G']
A2FE3VHELKQEJH ['B001GXRQW0']
A1LAIAJBVEX914 ['B00H5

### Look at the estimate of how a specific user would rate a specific item

In [7]:
#predict rating for a single user
algo_item.predict(uid = 'ACRIUHXH5F6V8', iid = 'B001GXRQW0')

Prediction(uid='ACRIUHXH5F6V8', iid='B001GXRQW0', r_ui=None, est=4.317770405281618, details={'actual_k': 1, 'was_impossible': False})

In [12]:
#################
#user based (doesn't work for this dataset)
#################

#to use user-based cosine similarity
sim_options_user = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
#choose centered KNN as the algorithm for finding similar suers
#algo_user = KNNWithMeans(sim_options=sim_options_user)
# Train the algorithm on the trainset, and predict ratings for the testset
#algo_user.fit(trainset)
#predictions_user = algo_user.test(testset)
# Then compute RMSE
#accuracy.rmse(predictions_user)

## 2.2 Model Based Approach

### Singular Value Decomposition (SVD)

In [13]:
#######################################################
#model based approach
#######################################################

###################
#SVD
###################
print('Using SVD')
#select the algorithm
algo_svd = SVD()
#fit the training set into the algorithm
algo_svd.fit(trainset)
predictions_svd = algo_svd.test(testset)
#then compute RMSE, MSE and MAE metrics of the test set
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_svd)
accuracy.mse(predictions_svd)
accuracy.mae(predictions_svd)

Using SVD
The testing accuracy in RMSE is: 
RMSE: 1.2132
MSE: 1.4717
MAE:  0.8873


0.8872699996099159

In [9]:
pred_df = pd.DataFrame(predictions_svd)
pred_df[['uid', 'iid', 'est']]

,uid,iid,est
0,A2YV8GR7IWAQ41,B00GOLGWVK,4.009031
1,AWFVKSW9BSTSX,B00BXLVAD6,4.557406
2,A2N3BP1J6X8WW3,B01E4QQ6QM,4.947137
3,AABEGIPYNQAEA,B009110O8C,4.547595
4,APY7B69YN6BUC,B0091JKJ0M,4.677635
...,...,...,...
29434,A1J4O0VR3G4D3I,B00A4EK106,4.771485
29435,A1O6VJM0XQ4K8C,B004LLILM8,4.699922
29436,A1GSJ508Q1SKZC,B00BXLTR6S,3.446075
29437,A1RMZXDJ0PC2B0,B00CHSWJPC,4.846729


In [15]:
#recommend top 10 items based on SVD
top_n = get_top_n(predictions_svd, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A2ESPSG38EUXJH ['B005EISPOG']
AS2L1TBZAFBZA ['B00JDQKFPC']
AP954LCNXA4VL ['B00CHQ7I2S']
AYXKDKN6Z4YLO ['B00P8N415E']
A21K4WXWNFXIEX ['B00OH7XY0U']
A1ZYK53V5UOWGG ['B01GP1W4LA']
A1G6JQDSB1JZEL ['B0062ONJ6U', 'B00B2TFNIW']
AFOQZF80Z8H6R ['B00MV9P8MS']
AX8FL2E1EVS70 ['B00MV9FM9C']
A1SNA99WLKLBLJ ['B00G4IV2VI']
A2J04RSYCLCZI1 ['B00JJ59UA8']
A20D1NHI6GLOF ['B00MN5VAWC']
A35SK1YPKYBDG1 ['B00VTVA9TQ']
ADYHB65IVT5NW ['B016J1LGPG']
A25ZQN76OXMOQ2 ['B00B2TFSO6']
A391REGDRKPKFB ['B005ESMJCU']
APG4S6DURMGJ ['B0066AZGD4']
A3LXT46950HJCT ['B00H5BMQEA']
A2NXTT2T3ZJPP2 ['B004Q7CK9M']
A3AYKKBPMYF8OQ ['B004LLIKZ6']
A2AH6UPEYEH8KL ['B005ESMMWW']
A2QR1PIISCOWCI ['B00I542CPE']
A1ZX7PV3HSOMJN ['B00AF0K90Q']
A13ASSBAX6F20E ['B00AF0K90Q']
A3EB0W0V266ZJN ['B01B25M52W']
ABMZCBSZG8V8J ['B004LLJ6XG']
A2210I20HOO8JH ['B00H5BMIHA']
A1FLEUK5PQC5SH ['B004LLIL2I']
A2ZKPDZBGU8QX0 ['B00CHSWGR8']
A25QL62FF1X8GK ['B00B2TFSO6']
A228QH2L3GOCZP ['B00BXLTJ8O']
A2B2520AGFT7JG ['B00KV30IHG', 'B00MN5VAWC']
A309PTIDLBGDJO ['B004L

A1FJY6VNX43UBS ['B00JDQJZWG']
A1DXXZZTYPBAC6 ['B00I542DM6']
AL6C4MOP0VCMT ['B004LLIKY2']
A18HKMAA9D1GKB ['B01E4QRAX0']
A1UG37FS1Y5IH6 ['B001GXRQW0']
A2RXZ1QAYAUXXY ['B0062ONJ6U']
A2IELBZJLJKYW0 ['B00GU2ZZ8Y']
A3N8AO29ZZG8M8 ['B00P9VF3DE']
A2BFPYCA2I7YOB ['B00C5UMB7S']
AGKLGL0NVP1J ['B005ESMF5G']
A2ADWMFSLW4L58 ['B0062ONAM8']
A11LUA9TARFAK4 ['B01E4QPDV6']
A32O77BN8KK54O ['B004LLILGO']
A3KMF710X3Y98U ['B01FERQPAM']
A30G91SEXFTVNK ['B00BXQIGA6']
A3OQWSQXYATL1V ['B005ESMMWW']
A2V38GYJJH3L32 ['B004LLIKVU']
ATAT9SGQ9JAT4 ['B004KNWX76']
A2QGYEDMBESQWG ['B00AF0K82U']
A2XBQQN9O3C4Y2 ['B0062ONJRE']
A12R29XOO1S4LC ['B014S1KPKG']
A1F069G5O3D7G4 ['B014S24DAI']
A2GDP0NRSAHXC2 ['B00CHSWKEM']
APPI8V93DBW27 ['B00H5BNLUS', 'B00B2TFRNI', 'B004KNWWU4', 'B00H5BMIHA']
A2ABIZUKM9B1Q7 ['B01B25LU7S']
A1C6UVRV5PH1OS ['B004Q7CK9M']
A174J1LSUZNT2Q ['B00GRLUECA']
A2LPJ0T3X1UVU1 ['B00H5BMF00']
A19WEEGB05YAZ3 ['B00BXLUO1U']
A2F0KYRZ2CH3WP ['B00CHQ7I2S']
A2LDP5Y6RSNEEW ['B004LLIKVU']
AII1AJQX884ZV ['B004LLIKVU']
A162

A38KUO7EKLQPW ['B00C5UMHOU']
A1138UIXWPX63A ['B004Q7CK9M']
AMNWAWG0KIL5H ['B00JDQJZWG']
A1EXTBJUUQHSYI ['B0091JKY0M']
AHIOLBJWJ9POH ['B00ET5WBU4']
A26H1FNT3Q9XLR ['B0145WHUP6']
ABH5AHMM5CA69 ['B0091JL3OI', 'B00BXLW4Q8']
A35JPJG80MIXNY ['B004KNWXA8']
A1YDI2QNLGEHFN ['B01CZ54L66']
A3AWPDBQLK3SZ8 ['B00B2TFSO6']
A2W8OTPDAZQMAY ['B00H5BMH44']
A2JCFTXD70NA8A ['B00H5BMZ32']
A2N5EBCZX885B3 ['B004LLIL32']
AUZEKYIXYGVOV ['B00A4EK69C']
AM53UDR5GE8GF ['B00AR51Y5I']
A1479Y2ETC7KVC ['B004LLIKVU']
A3FIQ5P8JSNF23 ['B0188LO3UK']
ANKKUIW6N8GTL ['B00H5BM3L6']
A2PVK0A93UYIJ3 ['B00JDQKFPC']
A2USYUX1VNWF1G ['B00GU2ZZ8Y']
A3U7GAPSU7MKDH ['B007HR1JWC']
A2D6KB1NYIS91J ['B00C5UMD6M']
A8X8EQZYJY1JI ['B00O3YIABA']
A33Q3LW155ACCT ['B00XU7O7OK']
A2R1U5UUJQYZTO ['B004LLILFK']
A1UHSMZEXVS0N1 ['B00CHSWG6O']
A3S598MY9GA4V3 ['B007V6ETXA']
AEJ27950VWWJV ['B00MV9H6VY']
A37V637ALK0MO9 ['B00BXLV9NM']
A30QBE1N177FR8 ['B00FTGFFOG']
A2UL1MSO6CT80 ['B00BXLVHZM', 'B00BXLW5QC']
AD4Q468IQ5D5B ['B00PG8502O', 'B01H5PPJT4']
AEXHXWC4Y

A1KGHKLYTWC5XD ['B00B2TFSO6']
AOUJI10YJGL6F ['B00BWDGY0E']
A3353A5G99R9QI ['B004W8D0Y4']
ASEOLDRQBWFOT ['B00JDQJZWG']
A218OIRAX594UR ['B00GXSCYVQ']
A2Y0PSNL692D43 ['B014S24DAI']
A1K1KXCIWQ2Y4Q ['B00EPLSUWA']
A30KNHVUO01B3J ['B00H5BMH44']
A8JQ06KSNXVJ7 ['B00H5BMH44']
A3TJLN982VQAF6 ['B004LLIL5A']
A3KTVL0V02PLFK ['B01E4QS95I']
AJS9GIRJOOD9D ['B006PJHPV2']
AQ0TLRNA53BSF ['B004LLILGO']
A1YPP7C4YVFT8M ['B00BXLTUBA']
A31OWF8SNFR2GQ ['B004Q7CK9M']
A33L4Y0ONHERP0 ['B00JDQJZWG']
A2BB6IHBL0P8KO ['B004LLILGO']
A2DHRL3NLT4YXZ ['B00F56PFYK']
AI8LBKJ3VONHC ['B01E4QS95I']
AXVMXUIB7AACE ['B0080IR6BA']
A3KE2PLRTT2W5R ['B00AF0K90Q']
A16M3TRNEAY0XP ['B00CRQ4CF4']
A330870L0YM026 ['B00BSXWOAC']
A2MZ511F4767WD ['B0078EPBHI']
A182NX50N1CWNU ['B00X4RERY6']
A17NPW3MNX3YP6 ['B01E4QS95I']
A10R323RXRECM8 ['B00KV30IHG']
A1WWKUWOHOMOWC ['B004KNWWV8']
A10X8H3IYIBFZP ['B00OQT3XUK']
A3S5XQLIM45U0L ['B004LLIL5A']
A3NESYX211M3DF ['B00AR51Y5I']
ARPYFO1JO3H1Q ['B004KNWX6C']
A1WX0HRUEJKYU2 ['B01E4QS95I']
A33GTBBC0J9ZEH ['B

A2YY7372EY4CJM ['B00BXLVFAO']
AEKIX574TIROI ['B004KNWX3U']
ANROJSRZGZT4B ['B00MV9P454']
A3H90NEHY1BHA0 ['B00BSXWOLG']
AEVKVN9958KPE ['B007V6ETDK']
A2OFTYKKBSBQAG ['B0091JKVU0']
A1MQXHYIN3DTOO ['B00Q5BN99C']
AOJJJPHA8THK ['B004Q7CK9M']
A1491TK7S0X4QF ['B01FERQPAM']
A3Q90TM0NNEVHZ ['B00ELQD11E']
A2884O10DUV3SN ['B00BXQIGA6']
APOJUN0JWXN7P ['B0091JKVU0']
A3GT6UEDEIQD1X ['B00O3YJMS0']
A3A8IB8UKLOOZ2 ['B0145WHSDA']
A1GQZ9GW1PLAUJ ['B00FTGTIOE', 'B00P3DUFRW']
A3AC171A213KZW ['B00B2TFSO6']
AQ8H9SFFP3PUG ['B0091JKY0M']
A2NHZ40TOJAX81 ['B004LLIKVU']
AACEZAH71JPYN ['B00H5BMZPU']
A3LEY2ROU2FFYA ['B00FTGSBBA']
AUIS8KORJ38V2 ['B004LLIL2I']
A28RVE4UTYN062 ['B014S1KPKG']
A3LYAS7V30E2UH ['B005ESMJ02']
A12XV63OZSJ6TY ['B00C5UMB7S']
A320DLHQTRFL5L ['B00CHQ7I2S']
A3AZ0Z2W4P7LUT ['B0091JKY0M']
A1MHLNANJ0CN0 ['B00BXLSGHO']
A374TAWW0ZUA14 ['B01E4QPDV6']
A1N2SYIVVNEN53 ['B00YD567IW']
A3W4FDXS9ZKMS0 ['B0145WHUP6']
A263FDYNKAUIOA ['B0091JKJ0M']
A2T851XNIXOFKD ['B004LLIL4G']
A3S28U0Y7972XW ['B00B2TFSO6']
A2WD0O

...

AJCQ5CSS05LOB ['B0091JKVU0']
A1K4EZZIFOC4XY ['B0145WHTES']
A18PBZSTOLJJR1 ['B00H5BMH44']
A22TK40WHZETWS ['B004Q7CK9M']
A3O897O7NK1WWD ['B004LLIL2I']
A13E8WFRRZDU7W ['B01FERR1VE']
A377Y9XI5U55M0 ['B00BXLTUBA']
A2BK0JAPQGP4Z8 ['B00GXSC31W']
A2NKFLP2542PM4 ['B004LLIKVU']
A2DFPYGAO2V2CT ['B004LLIKVU']
A4BTKEDZJF9A2 ['B01DWOZKSC']
A2E9B6X8BH1J9 ['B00CHSWG6O']
A3JYWN81ASVB3L ['B00MN5VAWC']
A3QR7P6VJDO4Q1 ['B004LLIKY2']
A3BTWI9VPBV293 ['B004LLIKY2']
A6UFO4TRTOZJK ['B00H5BMH44']
AXXUSXUA7LBLX ['B005ESMFP6']
A1SFJV0Q51M9CA ['B00ACA3UB4']
A1HSIDDJIBPITJ ['B005ESMMWW']
AI4ULDGBK0AK9 ['B004LLIKVU']
AVI6UR9TM8A20 ['B00S5JE4PU']
A16B3ERJHN268U ['B016PARP24']
A3T4AHISMM3QW4 ['B00PMOSVW8']
A105KC1B9FV1HT ['B005ESMMWW']
A3A5SMF949HOSP ['B0078EPBHI']
ANR7RQ0USZUAD ['B00B2TFSO6']
A3P2IPL3SKORQY ['B00FTGSBBA']
A1LSZEYWBVBSK8 ['B00OGPXG9W']
A33ZMZK3IAGLLA ['B00BSXWNXK']
A7Z8OVTEOI9R ['B0091JL3OI']
APB6NMAGI9VUF ['B00CXZPG0O']
A65KQ9T6PHQXZ ['B00BXLTUBA']
A3CA53SHC0DADH ['B01B25M52W']
A12YXCKFQZVYJX ['B00B2

A356WN2ED9M7IE ['B0066AZGD4']
A206C0ZB6JIRP7 ['B005ESMF5G']
A77P5DI121725 ['B00BT1XFBU']
A1Z1XEMJ50HK7P ['B01E4QQ6QM']
A2ZS8JC7A2BQ2N ['B004Q7CK9M']
A7W82JXXBPCXB ['B00BSXWOLG']
A5836551IOCP4 ['B00HI3QPWE']
ASY1R0LPJTU8T ['B007V5WJOM']
ATMLQYG9S8GWD ['B00OR54DES']
A2EPTRDTS9NBYC ['B004Q7CK9M']
A3FDSWWEXS6ZBQ ['B0091JKPDI']
A3OMLZ7CPYASMO ['B007V6ETDK']
AAPO9RVDHWWPW ['B00AR51Y5I']
A5XOH30FX9853 ['B00AR51Y5I']
A354UWRMKXQ7OA ['B004LLILGO']
A3RTZSE86K8VN9 ['B005EISOUQ']
A3AF6SR970XMZW ['B00MV9N316']
A1VONCTYY8PLT9 ['B00JDQJZWG']
A33444KQUZYOUS ['B00GXSC31W']
A3B0G76IJEB4NI ['B0091JKJ0M']
A3UUXFFTHH885I ['B0091JKY0M']
AJZNEW90PD59E ['B00BWDGYL8']
ARKV7QS93CLG9 ['B004Q7CK9M']
A3VI1BP0DCH1YQ ['B005ESMMWW']
AYK715QZEJ3ZL ['B00B2TF61G']
A23A1AVBH2OE4P ['B005DHN6E2']
A2F9YOEKZ882D3 ['B0066AZGD4']
A1WZJ0GT1KC52C ['B00H5BMH44']
A2GZKCD6RNGSOE ['B00G4IWEZG']
A2E6ZM4F7LDBNB ['B004KNWWU4']
A3MCGUE3S4HPL5 ['B004LLILK0']
A36SCWD0JAZQR9 ['B004LLIL5U']
A1UZNLXI8ZYVOF ['B004KNWX0S']
A2WJO6X7MOZQQG ['B00

A3132JOAMYZ3GT ['B0091JKVU0']
A2RT4ZW670BDDE ['B00G4IW938']
A3B6HHC6B9TXLU ['B016PARLMS']
AKCNNIMK2LO63 ['B0091JL3OI']
A2DYTAZJ28WWRH ['B01E4QPDV6']
A2DLKKB9XGVI61 ['B004Q7CK9M']
A2F6BUJRBEHMW ['B01E4QS95I']
A3BYAPX2CE9TD3 ['B0187SVTQU']
AGF5ZZSL4805O ['B00MV9LSHC']
AZOL8E8VNFVB1 ['B00H5BMQEA']
A2CX5RF8I3HSDA ['B005DHN6E2']
A147ETTQALMA8U ['B0091JKY0M']
ATS4VAZ4FCLRI ['B00IGYPALG']
A3QC2QWRN1ANEK ['B006PJI03Y']
A1EFXEUNMS3WUQ ['B00PMLDNBA']
A25I7GK07KNAOZ ['B00MVA457Q']
A3OR5NZZPBWX6Y ['B00H5BO5VW']
A1E1YQFJZVQQG9 ['B00BXLVAD6']
A3ESAKYAN8MB20 ['B004LLIKVU']
A36XQSHJU7CDZQ ['B00GXSCODO']
A30Z6SGVKL9DWT ['B004KNWWV8']
A3751RR675CGEY ['B004LLIKVU']
APHOLSH1PZHV6 ['B00ACA3UMS']
A26S58QFM8LT2Y ['B00G4IV2VI']
A2T70UQV3SWYV3 ['B00H5BMZPU']
AUYUJ7PDISP3K ['B00PMLDLKI']
A2CVU5FL3F6VYR ['B00BXLUI5M']
A19ANIDSGLJP4M ['B00BXLURUI']
AU4EHS29GRFKI ['B007V5WJOM']
AFEC6Q1OKJS2B ['B0091JKVU0']
A1B0I1PNBHWX8G ['B00MV9OO2S']
AV48FXX8WEZ05 ['B01B25LFGY']
A1HUK5KGPVD2P9 ['B00CRQ496G']
A3LP0DAPOZ3XY0 ['B00

A3H6UHMR24N8Z5 ['B00BT1XFBU']
A1NF10T1BGPQQ8 ['B004LLIKVU']
A2KT4PXQWC0F4G ['B004KNWWU4']
AN2FMIKTNVXH1 ['B005ESMMWW']
A2S5GLNZA9BKSO ['B0091JL3OI']
A3O6D1XEXXYU1K ['B00CRQ4URY']
AXG98JLOB1QJV ['B00JDQJZWG']
A2DP95JUY37VN ['B004LLIL3C']
A3BRZ61SA7K7G6 ['B004KNWWTK']
A2N70K8SYOWPSL ['B00AR51Y5I']
A1B6NT7IFXO6I7 ['B004WKPV3K']
A1OCYEWYFLB767 ['B014S24DAI']
AQGDN28LPCLY1 ['B00H5BNLUS']
A1NN1GEPORBL1C ['B00JDQKFPC']
AV89SOAWZEMU5 ['B007EE5I6G']
A2LB4X9SP24J77 ['B0091JKVU0']
A3IXWTVV640NEC ['B00H5BMH44']
A2BTFV34VI6MAP ['B006PJHPV2']
A3IKWB6CGF2B3T ['B00CHSWJPC']
AC5KO3P90B8S5 ['B004LLIKY2']
A29QTODE67FOYD ['B0091JL3OI']
APMYTUORLKOAJ ['B00G4IWEZG']
A2ZF7V02HGXEH3 ['B004LLIL0K']
A2MD4M1RDBRM17 ['B01E4QS95I']
A297RF4K8FHYRA ['B004Q7CK9M']
A332S4LHX5RLFN ['B004Q7CK9M']
A4JWU1T5U20Z4 ['B00CHSWJPC']
A14BDPQC9ESAYY ['B0069VHL5Q']
A1EOMMDWHCDEZ5 ['B00H5BM23K']
AP7GFXZLIYSZ5 ['B004Q7CK9M']
A1PLQLOQQ5PETT ['B00AR51Y5I']
A2DFKVXZE4R6WO ['B00CRQ47WC']
A33SSICDN6X93C ['B004Q7CK9M']
A20PMAG3JX3Y2N ['B0

A38MZ8QE5EI7B0 ['B004Q7CK9M']
A13T12UGTVBNNH ['B00CHSWJPC']
A2Y58DGXMJTNQR ['B00MV9FCFQ']
A2Y1WK2C5KX2WC ['B004KNWWMW']
A3IK4WYA5Y5ZOA ['B004Q7CK9M']
A24VQNSOIBUW5G ['B004LLIKVU']
A1O8K65XAMZFCX ['B00BXLSIJ0']
A3VU19Y4V0AG85 ['B007V6ETDK']
A3M1W0EG8P7Q9R ['B014S24DAI']
A1PSE8FEIWY221 ['B00OX9OKO6']
A1BF5030JMHQ7F ['B00GOLH84U']
A17IXQ1O9SWSYQ ['B00C5UMHOU']
A1MJGP2PF8IRF5 ['B0091JKVU0']
A1IPZ25DHWUK4A ['B00BXLW4Q8']
A30NXXF341WYMI ['B00CHSWG6O']
AMQS22UB5JM4N ['B0066AZGD4']
A38LAQCNE0CDO7 ['B00BXQIGA6']
A9ODQSFA8NR3N ['B00I5426SM']
A18GRE0NLGKP7U ['B0066AZGD4']
A6QJNVHOHUYM5 ['B004LLIKVU']
A10SD85N5IT7DL ['B00FTGTIOE']
A2K3L7UECH8F0E ['B005ESMJCU']
ARCHUPU7ACYEX ['B0091JKVU0']
ANF2J3EZM50DL ['B014S24DAI']
AG02PU3A5EEKQ ['B00MV9FCFQ']
AIOVI6ORDW3HU ['B0062ONA9Q']
A2KUUBOJRHTP04 ['B004KNWWU4']
A4KRB9TKRU2T2 ['B00BT1XEB6']
A3MQO7Q3Z7ZF1D ['B00H5BM3L6']
A3EOQCI4TJRLS0 ['B00YD55RD8']
ATSQCGKT2FDLE ['B00ELQD11E']
ARKTDYFG0T4EV ['B00I5426SM']
A1N6KFJJ7LWMB2 ['B014LMWGCM']
A2G0MYD0GDEJJE ['B00

In [16]:
#predict rating for a single user
algo_svd.predict(uid = 'ACRIUHXH5F6V8', iid = 'B001GXRQW0')

Prediction(uid='ACRIUHXH5F6V8', iid='B001GXRQW0', r_ui=None, est=4.668296619588155, details={'was_impossible': False})

### Alternating Least Squares (ALS)

In [12]:
###################
#ALS
###################
start_time = time.time()
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo_als = BaselineOnly(bsl_options=bsl_options)
#fit the training set into the algorithm
algo_als.fit(trainset)
predictions_als = algo_als.test(testset)
#then compute RMSE, MSE and MAE metrics of the test set
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_als)
accuracy.mse(predictions_als)
accuracy.mae(predictions_als)

print("--- %s seconds ---" % (time.time() - start_time))

Using ALS
Estimating biases using als...
The testing accuracy in RMSE is: 
RMSE: 1.2187
MSE: 1.4853
MAE:  0.8959
--- 2.320751428604126 seconds ---


In [22]:
predictions_als

[Prediction(uid='A2ESPSG38EUXJH', iid='B005EISPOG', r_ui=5.0, est=4.793260475847529, details={'was_impossible': False}),
 Prediction(uid='AS2L1TBZAFBZA', iid='B00JDQKFPC', r_ui=5.0, est=4.661729137506447, details={'was_impossible': False}),
 Prediction(uid='AP954LCNXA4VL', iid='B00CHQ7I2S', r_ui=5.0, est=4.704042330573312, details={'was_impossible': False}),
 Prediction(uid='AYXKDKN6Z4YLO', iid='B00P8N415E', r_ui=4.0, est=4.750608120886235, details={'was_impossible': False}),
 Prediction(uid='A21K4WXWNFXIEX', iid='B00OH7XY0U', r_ui=1.0, est=4.402787062178361, details={'was_impossible': False}),
 Prediction(uid='A1ZYK53V5UOWGG', iid='B01GP1W4LA', r_ui=4.0, est=3.5939534494276084, details={'was_impossible': False}),
 Prediction(uid='A1G6JQDSB1JZEL', iid='B00B2TFNIW', r_ui=4.0, est=4.782113634759634, details={'was_impossible': False}),
 Prediction(uid='AFOQZF80Z8H6R', iid='B00MV9P8MS', r_ui=5.0, est=4.5517410740617965, details={'was_impossible': False}),
 Prediction(uid='AX8FL2E1EVS70', i

In [18]:
#recommend top 10 items based on SVD
top_n = get_top_n(predictions_als, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A2ESPSG38EUXJH ['B005EISPOG']
AS2L1TBZAFBZA ['B00JDQKFPC']
AP954LCNXA4VL ['B00CHQ7I2S']
AYXKDKN6Z4YLO ['B00P8N415E']
A21K4WXWNFXIEX ['B00OH7XY0U']
A1ZYK53V5UOWGG ['B01GP1W4LA']
A1G6JQDSB1JZEL ['B0062ONJ6U', 'B00B2TFNIW']
AFOQZF80Z8H6R ['B00MV9P8MS']
AX8FL2E1EVS70 ['B00MV9FM9C']
A1SNA99WLKLBLJ ['B00G4IV2VI']
A2J04RSYCLCZI1 ['B00JJ59UA8']
A20D1NHI6GLOF ['B00MN5VAWC']
A35SK1YPKYBDG1 ['B00VTVA9TQ']
ADYHB65IVT5NW ['B016J1LGPG']
A25ZQN76OXMOQ2 ['B00B2TFSO6']
A391REGDRKPKFB ['B005ESMJCU']
APG4S6DURMGJ ['B0066AZGD4']
A3LXT46950HJCT ['B00H5BMQEA']
A2NXTT2T3ZJPP2 ['B004Q7CK9M']
A3AYKKBPMYF8OQ ['B004LLIKZ6']
A2AH6UPEYEH8KL ['B005ESMMWW']
A2QR1PIISCOWCI ['B00I542CPE']
A1ZX7PV3HSOMJN ['B00AF0K90Q']
A13ASSBAX6F20E ['B00AF0K90Q']
A3EB0W0V266ZJN ['B01B25M52W']
ABMZCBSZG8V8J ['B004LLJ6XG']
A2210I20HOO8JH ['B00H5BMIHA']
A1FLEUK5PQC5SH ['B004LLIL2I']
A2ZKPDZBGU8QX0 ['B00CHSWGR8']
A25QL62FF1X8GK ['B00B2TFSO6']
A228QH2L3GOCZP ['B00BXLTJ8O']
A2B2520AGFT7JG ['B00KV30IHG', 'B00MN5VAWC']
A309PTIDLBGDJO ['B004L

AL6C4MOP0VCMT ['B004LLIKY2']
A18HKMAA9D1GKB ['B01E4QRAX0']
A1UG37FS1Y5IH6 ['B001GXRQW0']
A2RXZ1QAYAUXXY ['B0062ONJ6U']
A2IELBZJLJKYW0 ['B00GU2ZZ8Y']
A3N8AO29ZZG8M8 ['B00P9VF3DE']
A2BFPYCA2I7YOB ['B00C5UMB7S']
AGKLGL0NVP1J ['B005ESMF5G']
A2ADWMFSLW4L58 ['B0062ONAM8']
A11LUA9TARFAK4 ['B01E4QPDV6']
A32O77BN8KK54O ['B004LLILGO']
A3KMF710X3Y98U ['B01FERQPAM']
A30G91SEXFTVNK ['B00BXQIGA6']
A3OQWSQXYATL1V ['B005ESMMWW']
A2V38GYJJH3L32 ['B004LLIKVU']
ATAT9SGQ9JAT4 ['B004KNWX76']
A2QGYEDMBESQWG ['B00AF0K82U']
A2XBQQN9O3C4Y2 ['B0062ONJRE']
A12R29XOO1S4LC ['B014S1KPKG']
A1F069G5O3D7G4 ['B014S24DAI']
A2GDP0NRSAHXC2 ['B00CHSWKEM']
APPI8V93DBW27 ['B00B2TFRNI', 'B00H5BNLUS', 'B004KNWWU4', 'B00H5BMIHA']
A2ABIZUKM9B1Q7 ['B01B25LU7S']
A1C6UVRV5PH1OS ['B004Q7CK9M']
A174J1LSUZNT2Q ['B00GRLUECA']
A2LPJ0T3X1UVU1 ['B00H5BMF00']
A19WEEGB05YAZ3 ['B00BXLUO1U']
A2F0KYRZ2CH3WP ['B00CHQ7I2S']
A2LDP5Y6RSNEEW ['B004LLIKVU']
AII1AJQX884ZV ['B004LLIKVU']
A162PJT7NRGOJL ['B01E4QUN0W']
A1RUIA07KQ5M3 ['B00G4IWEZG']
A9PUM

A2D6KB1NYIS91J ['B00C5UMD6M']
A8X8EQZYJY1JI ['B00O3YIABA']
A33Q3LW155ACCT ['B00XU7O7OK']
A2R1U5UUJQYZTO ['B004LLILFK']
A1UHSMZEXVS0N1 ['B00CHSWG6O']
A3S598MY9GA4V3 ['B007V6ETXA']
AEJ27950VWWJV ['B00MV9H6VY']
A37V637ALK0MO9 ['B00BXLV9NM']
A30QBE1N177FR8 ['B00FTGFFOG']
A2UL1MSO6CT80 ['B00BXLVHZM', 'B00BXLW5QC']
AD4Q468IQ5D5B ['B00PG8502O', 'B01H5PPJT4']
AEXHXWC4YC4Z9 ['B004LLIKVU']
A2BFFDEQYQTNXU ['B005DHN6E2']
A3CV3VHNP6FE6W ['B01GF6X20W']
A2CT8TRX7IVYR2 ['B002YEWXZ0']
A40M3CVVHNGN4 ['B005ESMMWW']
A2PECE0204H284 ['B005ESMJ02']
AOFR97L21BAKK ['B00JDQJZWG']
A3HCS0O648JUML ['B016PASHH6']
A3OHBLI9CHQIGI ['B005ESMMWW']
A177HLB30IAQNW ['B00PG23QFI']
A7Y9UC40Z1J2F ['B004Q7CK9M']
A1Z1IK7KMCUSHN ['B00H5BMRN0']
A3P05EA7VJEGP5 ['B004LLILPU']
A9S2JWGSSEM9G ['B00BXLUMGC']
AJNP6ABM23DT6 ['B004LLILK0']
AONN6P28UI3UU ['B004Q7CK9M']
A1P8SDDTS4WZU6 ['B00H5BMH44']
A35E06SVKH04KB ['B004LLIKVU']
ANNK79IVRD0T7 ['B00AF0K82U']
AR7I243JS5YC4 ['B00B2TFSO6']
A12D0DAOZ864VD ['B01H5PPJT4']
A13X19AIA4ZOY3 ['B005ESMF

AJS9GIRJOOD9D ['B006PJHPV2']
AQ0TLRNA53BSF ['B004LLILGO']
A1YPP7C4YVFT8M ['B00BXLTUBA']
A31OWF8SNFR2GQ ['B004Q7CK9M']
A33L4Y0ONHERP0 ['B00JDQJZWG']
A2BB6IHBL0P8KO ['B004LLILGO']
A2DHRL3NLT4YXZ ['B00F56PFYK']
AI8LBKJ3VONHC ['B01E4QS95I']
AXVMXUIB7AACE ['B0080IR6BA']
A3KE2PLRTT2W5R ['B00AF0K90Q']
A16M3TRNEAY0XP ['B00CRQ4CF4']
A330870L0YM026 ['B00BSXWOAC']
A2MZ511F4767WD ['B0078EPBHI']
A182NX50N1CWNU ['B00X4RERY6']
A17NPW3MNX3YP6 ['B01E4QS95I']
A10R323RXRECM8 ['B00KV30IHG']
A1WWKUWOHOMOWC ['B004KNWWV8']
A10X8H3IYIBFZP ['B00OQT3XUK']
A3S5XQLIM45U0L ['B004LLIL5A']
A3NESYX211M3DF ['B00AR51Y5I']
ARPYFO1JO3H1Q ['B004KNWX6C']
A1WX0HRUEJKYU2 ['B01E4QS95I']
A33GTBBC0J9ZEH ['B0066AZGD4']
A7AABT2JJU4IH ['B00H5BNNY2', 'B00I5428EE']
A2ZOVEFDMIIQB3 ['B0091JL3OI']
A278X0K3CH31FU ['B00MV9FCFQ']
AI4ZECPIA2GBL ['B005ESMJ02']
ARHHRAD2N15WL ['B004LLIKY2']
A1QUXK6W7ZWSVL ['B0187SVTQU']
A1PO17BY8LMGIC ['B00CHQ7I2S']
A32AP8GXVJFSQV ['B01E4QUN0W']
AMSMDHGYF9NXJ ['B004LLIKVU']
AMORNEG7L71JK ['B005EISPCS']
A8PWWV

A2NHZ40TOJAX81 ['B004LLIKVU']
AACEZAH71JPYN ['B00H5BMZPU']
A3LEY2ROU2FFYA ['B00FTGSBBA']
AUIS8KORJ38V2 ['B004LLIL2I']
A28RVE4UTYN062 ['B014S1KPKG']
A3LYAS7V30E2UH ['B005ESMJ02']
A12XV63OZSJ6TY ['B00C5UMB7S']
A320DLHQTRFL5L ['B00CHQ7I2S']
A3AZ0Z2W4P7LUT ['B0091JKY0M']
A1MHLNANJ0CN0 ['B00BXLSGHO']
A374TAWW0ZUA14 ['B01E4QPDV6']
A1N2SYIVVNEN53 ['B00YD567IW']
A3W4FDXS9ZKMS0 ['B0145WHUP6']
A263FDYNKAUIOA ['B0091JKJ0M']
A2T851XNIXOFKD ['B004LLIL4G']
A3S28U0Y7972XW ['B00B2TFSO6']
A2WD0OTJYMLFDK ['B004Q7CK9M']
A1NDXGOBYIZO20 ['B004LLIKWO']
A1S7XAXT4HISSR ['B00KV30IHG']
A2HOAA0JK0ZUPL ['B005ESMMWW']
ANIRELKAL9FBK ['B00JDQKFPC']
A10SQ446Y9G6AW ['B00MV9H6VY']
AYCNAH04PZCKE ['B007V6ETDK']
A1U6KMQ6SI1FKG ['B00BXLVAD6']
A2GD55WK3P0SJQ ['B018T7G9XM']
A3S1ACNXVVJJPE ['B014S1KPKG']
A1BDVYL26LW1XD ['B004LLIKVU']
A3PV9IN31WERUB ['B004W8D0Y4']
A3FTQMOKW2MDEG ['B004LLIL5A']
A2AD5ZO1Q2O5GB ['B004KNWWMW']
A1VWREIKPF8U0Z ['B00H5BMH44']
AR8III3UJQVHW ['B0078EPBHI']
ANXVK6D3IC1UJ ['B00B2TFSO6']
A1XNCAKTHX08QM ['

...

AHCOPZ1PGSAIP ['B00FRL7G7M']
AO00HLU6BX31G ['B004Q7CK9M']
A3FZY8UDHESAHK ['B004LLIL5A']
A34VW2PKJ9ZS3C ['B00BXLV9NM']
A2NKP5I00YR9ZW ['B004LLIKY2']
AEZCQSQAO98SU ['B00K4PIE68']
A2ZGPXJSGWZNL2 ['B00G4IV2VI']
AFQ14NCM82MIR ['B00IGYPALG']
A2TOE4NZUT6GU0 ['B00MV9GRNW']
A94FAPY7ICVTC ['B00BWDGY0E']
A1TZOMCVIR6LB1 ['B00PG40RBW']
AY6Y6SNQUUFO ['B005ESMJ02']
A1Q6B2J857AW3D ['B00C5UMB7S']
AX8TL55KP2U3H ['B00BXLTR6S']
A297PKALHZ4KHK ['B004Q7CK9M']
A117ROYJJTDECF ['B0091JL3OI']
A20ILAI2AEA64O ['B018UPUJJI']
A1FYZHF9B5KXJB ['B007V6ETDK']
A2KAOIY8M4D044 ['B00H5BNIR4']
A116JUGPE39P6L ['B00BXLUMGC']
A2TUQVI13C8KJV ['B004LLIL5A']
A21DKFCRAVN8RZ ['B004LLIKVU']
A2CRDW6KNACG26 ['B0062ONJ6U']
A3CY9KNIZVEL8X ['B001GXRQW0']
A8BM5FG7D6VGX ['B014S24DAI']
A3P3E560J5GYEV ['B00KV30IHG']
A13NZVMMSJJ2Y2 ['B004KNWX4O']
AY8JCJQDGNITX ['B00H5BMH44']
A2GT0PMVAXZ40L ['B0091JL3OI']
A3C35TTK8MZBPR ['B01E4QRAX0']
AJX0CXILSFMLT ['B007V6ETDK']
A2I03TLMVTASA7 ['B006PJHP62']
A3HN9PMEGAM6JT ['B0062ONJRE']
A3ANM6KJTFIL ['B004Q7

A28FBF13MNRAAJ ['B01FERR9FW']
A8FVL2MYEIZLQ ['B018UPUJJI']
AWCI1WKGESX0I ['B004LLIL4G']
A29T1ONPP0IC7J ['B00AR51Y5I']
AOIK2JWRLMC5W ['B0091JKJ0M']
A2D9LS0VBXUP9Y ['B00O3YJ2TE']
AKKZSWPUB7444 ['B00H5BNIR4']
A2R819E0C80FXW ['B00BXLUO1U']
A18TGLOS1VL1MQ ['B01E4QS95I']
A1EKS3LLU2XXWL ['B00H5BMIHA']
A3I40BF54E8MDK ['B0066AZGD4']
A14NSERPA4150S ['B002DN7XS4']
A2YXNBPWX6J585 ['B00G4IV2VI']
A323VPJLM1LOT3 ['B004KNWWOA']
A3B188V77RHK7O ['B004LLIKVU']
A2MCPWDQSYG7RK ['B004LLIKVU']
A34EVFRKYQQY9X ['B0091JL3OI']
A2JBM22HNCSXTB ['B00BXLVAD6']
A1W6BILQJ1NX4W ['B0066AZGJI']
A1WMOMDFRJ5HE5 ['B0091JKY0M']
A820PDB8HCLPQ ['B004Q7CK9M']
A2CHJN0CAZFPHO ['B004RD8XPK']
A1KQ0W1B4AVIAX ['B00I5426SM']
A3VB68LNDOS2S1 ['B00F2RZMEA']
A306CE9AX3AGSZ ['B00H5BNRVQ']
A1CJB6JGZLUP8V ['B0091JKJ0M']
A3IC9MGVK7ABK4 ['B00BXLTZ6K']
A1S53QKJ8YFWIN ['B004LLIL5A']
AQ3QEWH509M3E ['B00CHQ7I2S']
A2VGP0BOWK1K8U ['B014S1KPKG']
A3BNO6LX944D6I ['B004Q7CK9M']
A1B3IF2BF7UM8K ['B00895H7FO']
A2N7V6NY8ZXGMA ['B014S24DAI']
A1MVHIZXTDBD7D [

A39FB6FAJ0CBOA ['B00H5BMH44']
ABH8DSULR8P90 ['B0091JKVU0']
A2EW34DK8RQ9KE ['B018T7WZFS']
A26I51P7LEOLXU ['B00G4IWEZG']
A1WZKH7SA4ZXVS ['B00MVA46R0']
A3M0X4EH9UDPHT ['B005ESMJ02']
ARL7FCR31WY7R ['B004KNWX3A']
A3EHJO48E90K0S ['B014S24DAI']
ATPZXGXV817M4 ['B00JDQJZWG']
AIXYBI37CBMYV ['B00A4EK69C']
A2SGR0JS4ATQLO ['B0091JKY0M']
A2R4RG3VDMCXU6 ['B00BXLTUBA']
A3IL88F1IA1EKA ['B005ESMMWW']
A3KBEYSXVROMWL ['B004LLILM8']
A3QAN96Y68ST8K ['B00H5BMH44']
AGN8485OZ9RA ['B00FTJI60I']
AYTP5E2SLM088 ['B005ESMF5G']
A3UJ6P8U7MW7QK ['B005DHN6E2']
A226EXHZOOWIKW ['B01E4QUN0W']
A1N6Y5KA7LBWLY ['B005ESMJ02']
A1Z1YCQQD7NMHR ['B0091JKVU0']
A15566VHMINJRB ['B004Q7CK9M']
A3OJJYV29104JF ['B01E4QUN0W']
A388B1XEL8RHRP ['B004KNWX3U']
A14WTBNXOM7P1E ['B004LLILGO']
A2Y44FGWGNBBDZ ['B00FTJI60I']
A3PVXNCY93NY9X ['B007V6ETXA']
A3GDDYHBURLBG7 ['B006PJI0Y8']
AWEN1QIQLY10O ['B00G3LBDDS']
A2H4VNM4QQ1G7F ['B00JDQJZWG']
A21Z1T8LDV76 ['B014S24DAI']
A1ZFRQDHV1O407 ['B0091JKY0M']
A3G5YW3TCF8W2G ['B0091JKPDI']
AGMXSCJNQKDHX ['B00C

A2ROHF6R87MYD9 ['B004LLIKY2']
A3P8WNTYNYF2DP ['B0091JKJ0M']
AMC5R7HY7ZTIA ['B00H5BMH44']
A1LCDJQDQK2V39 ['B00MV9GCYQ']
AGQHR9DR8VT26 ['B00BSXWOLG']
A1KQIVAC0MZKXZ ['B00H5BMF00']
A2BUS9RMVMQ9BP ['B01GKWEPBG']
A10ALY0DCTF752 ['B00B2TFSO6']
A33PRS239816ZU ['B005DHN6E2']
A1AJJ749D80VCY ['B00GXSC70E']
A3VB6RPL71HCGM ['B00Q5BOSYC']
A58Q57OYXZPIA ['B004KNWWU4']
ABIW6HTM5EQRE ['B00MV9FM9C']
A3NSHMD24JDKGG ['B00BXLSPGQ']
A1XGFIFI2CJQQO ['B00HLAO65M']
A1XJLZUPSS9BJ2 ['B004LLILPU']
A3FDHULM24T3DD ['B01E4QQ6QM']
A2W9QRJ7SQSMTP ['B004KNWX1W']
ABGTKKS8N8ACJ ['B004KNWX3A']
A6TJP9RKCOEK2 ['B00IGYPALG']
A2GL2SHYSNBFUT ['B00MV9OHUM']
AGP73GTO152O1 ['B00MV9N316']
A1DTDU0FHJHD6U ['B004LLIKVU']
A357WBUOUWB6PA ['B00BXLT1UA']
A13957WD8QU11P ['B004LLIL5A']
AS1UMIRTSW3FL ['B00CT777M6']
A10FASJXUJ45KC ['B00CXZPG0O']
A20PP83BV5YG8J ['B00BXLVFAO']
A14D3UGP9MFS9I ['B005ESMJ02']
A25GE1R6D38S65 ['B0091JKVU0']
A1ZN28T4U931VE ['B01FERR1VE']
A1D06XJ30UN00C ['B006PJHP62']
A37HIWGJ615675 ['B0091JL3OI']
A3FDQR5R4HCSJR ['B

A1JXE9RG9ROM9B ['B01E4QUN0W']
A2V1J3JT5OOZFO ['B014S24DAI']
A3AGWMV9E4B5JH ['B00A4EK69C']
A2C9QMOCOC2U9W ['B004KNWWTK']
A1IJI23BYK3LRR ['B016PARP24']
AK1IBFNAB83A2 ['B00PMLD91O']
A2RJ5V1Q7RYXET ['B004Q7CK9M']
A2BDMOO4PIE0N ['B00CHSWGR8']
A2N346MFSXCVQI ['B00ACA3UMS']
A1808PMNJHYMLL ['B00AF0K90Q']
ACDK1RUI89MJZ ['B00BXLUI5M']
A52I97FM435W5 ['B004Q7CK9M']
A34OW9YN7H1BBV ['B004LLIKVU']
A25L9OTBG6QJCU ['B0188LO3UK']
A3GBN87NQVRJNU ['B005ESMJ02']
A1P2MBJ6BCAFO5 ['B00F56PFYK']
A18OOGLFGAGEI5 ['B004LLIKVU']
A20S8FY6H48UCQ ['B004Q7CK9M']
AM55IO81M9DFN ['B004KNWWZY']
A3TUE266SG8K4D ['B005ESMF5G']
A1DYA3C9ZYXBDI ['B00I5428VM']
A1QI9TIJFJ3DLX ['B00EPLT7NQ']
A1EU5WVQG95UC7 ['B004LLILEG']
ASMTS7RUU4X7O ['B00H5BMF00']
A3GIDO9O8PHDAS ['B00JDQJZWG']
ADS8BAC8FLYA5 ['B00PG40SDY']
AK2XYFZ078QHG ['B00JDQKFPC']
A12YUGHOHQYCVE ['B0091JKVU0']
A10C2LNRK4SMJL ['B006PJI0Y8']
A4DQSV3T2EVIP ['B004Q7CK9M']
A1KQLYM1EO2VP0 ['B005EISP3M']
A30ZHN9WZ1IHKO ['B004LLIL5A']
A3BZAHYJ1UIIEU ['B0078EPBHI']
A2RF9VB57320MN ['B0

In [19]:
#predict rating for a single user
algo_als.predict(uid = 'ACRIUHXH5F6V8', iid = 'B001GXRQW0')

Prediction(uid='ACRIUHXH5F6V8', iid='B001GXRQW0', r_ui=None, est=4.209152849124371, details={'was_impossible': False})

In [20]:
pred_df = pd.DataFrame(predictions_als)
pred_df[['uid', 'iid', 'est']]

,uid,iid,est
0,A2ESPSG38EUXJH,B005EISPOG,4.793260
1,AS2L1TBZAFBZA,B00JDQKFPC,4.661729
2,AP954LCNXA4VL,B00CHQ7I2S,4.704042
3,AYXKDKN6Z4YLO,B00P8N415E,4.750608
4,A21K4WXWNFXIEX,B00OH7XY0U,4.402787
...,...,...,...
29434,A2I2N1P6Z2Y0XR,B00JDQJZWG,4.762448
29435,A25AEP2S1I6LMP,B004Q7CK9M,4.617488
29436,A4PJ46I4SNTYL,B004LLIL6Y,4.869426
29437,A1X8D2AL3TM73B,B00G4IUURA,4.462292


In [21]:
#grid search cross validation, not necessary for our project
#param_grid = {
#    "n_epochs": [10],
#    "lr_all": [0.01],
#    "reg_all": [0.5]
#}
#gs2 = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=5)
#gs2.fit(data)

#print(gs2.best_score["rmse"])
#print(gs2.best_params["rmse"])
#print("--- %s seconds ---" % (time.time() - start_time))